In [29]:
import pandas as pd
import numpy as np
import seaborn as sns
import spacy
import string
import regex as re
from unidecode import unidecode
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score
from keras.callbacks import EarlyStopping
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

In [43]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd

## CONFIG - POSTGRES ##
db_config = {
    'user': 'postgres',
    'password': 'admin',
    'host': 'localhost',
    'port': '5432',
    'database': 'fluffyapi'
}

try:
    # Testar a conexão usando psycopg2
    conn = psycopg2.connect(**db_config)
    print("Conexão bem-sucedida")

    # Consultar a tabela 'review'
    query = "SELECT * FROM review;"
    df_review = pd.read_sql_query(query, conn)

    # Exibir os dados da tabela
    print(df_review.head())

    # Fechar a conexão
    conn.close()
        
except Exception as e:
    print("Erro ao conectar ao PostgreSQL:", e)


Conexão bem-sucedida
Empty DataFrame
Columns: [review_id, review_comment_message, review_score, predictions, geolocation_lat, geolocation_lng, geolocation_state, geolocation_country, review_creation_date, origin, geolocation_point, creationdate]
Index: []


In [44]:
df_review

,review_id,review_comment_message,review_score,predictions,geolocation_lat,geolocation_lng,geolocation_state,geolocation_country,review_creation_date,origin,geolocation_point,creationdate


In [39]:
import psycopg2
from sqlalchemy import create_engine

## CONFIG - POSTGRES ##
db_config = {
    'user': 'postgres',
    'password': 'admin',
    'host': 'localhost',
    'port': '5432',
    'database': 'fluffyapi'
}
conn = psycopg2.connect(**db_config)
engine = create_engine('postgresql+psycopg2://' + db_config['user'] + ':' + db_config['password'] + '@' + db_config['host'] + ':' + db_config['port'] + '/' + db_config['database'])

In [31]:
reviews = pd.read_parquet('deploycsv.parquet')

In [32]:
reviews['review_id'] = range(1, len(reviews) + 1)

In [33]:
# Definir a sequência desejada de colunas
sequencia_colunas = ['review_id', 'review_comment_message', 'review_score', 'predictions', 
                     'geolocation_lat', 'geolocation_lng', 'geolocation_state', 'geolocation_country', 
                     'review_creation_date', 'origin', 'geolocation_point']

# Reorganizar as colunas do DataFrame
reviews = reviews[sequencia_colunas]

In [51]:
reviews['creationdate'] = pd.to_datetime('now')

In [58]:
reviews.to_csv('deploycsv.csv', sep=";")

In [61]:
reviews = pd.read_csv('deploycsv.csv',sep=";")

In [ ]:
reviews.dropna(columns="review_comment_message")

In [62]:
# Substitua 'nome_da_tabela' pelo nome da tabela em seu banco de dados
nome_da_tabela = 'review'

# Inserir os dados na tabela review
reviews.to_sql(nome_da_tabela, engine, if_exists='append', index=False)

print("Dados inseridos com sucesso na tabela", nome_da_tabela)

AttributeError: 'Engine' object has no attribute 'cursor'

In [63]:
import pandas as pd
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from unicodedata import normalize
import string

# Carregar lista de stopwords em inglês
stop_words = set(stopwords.words('english'))

# Função para remover acentuações e palavras pontuadas
def preprocess_text(text):
    # Remover acentuações
    text = normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII')
    # Remover palavras pontuadas e stopwords
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words and token not in string.punctuation and token.strip()]
    return ' '.join(tokens)

# Função para contar a frequência das palavras e aplicar o pré-processamento
def count_words(text):
    tokens = word_tokenize(text.lower())
    tokens = [preprocess_text(token) for token in tokens]
    return Counter(tokens)

# Filtrar o DataFrame para cada categoria de 'predictions'
predictions_0 = reviews[reviews['predictions'] == 0]['review_comment_message']
predictions_1 = reviews[reviews['predictions'] == 1]['review_comment_message']
predictions_2 = reviews[reviews['predictions'] == 2]['review_comment_message']

# Contagem de palavras e aplicação do pré-processamento para cada categoria
word_counts_0 = predictions_0.apply(count_words).sum()
word_counts_1 = predictions_1.apply(count_words).sum()
word_counts_2 = predictions_2.apply(count_words).sum()

# Selecionar as top 60 palavras mais citadas em cada categoria
top_words_0 = word_counts_0.most_common(60)
top_words_1 = word_counts_1.most_common(60)
top_words_2 = word_counts_2.most_common(60)

# Juntar os resultados em um único DataFrame
top_words_df = pd.DataFrame({
    'word': [word[0] for word in top_words_0 + top_words_1 + top_words_2],
    'counts': [word[1] for word in top_words_0 + top_words_1 + top_words_2],
    'sentiment': ['0'] * 60 + ['1'] * 60 + ['2'] * 60
})

# Exibir o DataFrame com as top 60 palavras mais citadas em cada categoria
print(top_words_df)

In [ ]:
# Substitua 'nome_da_tabela' pelo nome da tabela em seu banco de dados
nome_da_tabela = 'palavra'

# Inserir o DataFrame reviews no banco de dados PostgreSQL
top_words_df.to_sql(nome_da_tabela, engine, if_exists='append', index=False)

print("Dados inseridos com sucesso na tabela", nome_da_tabela)